In [ ]:
!pip install emoji

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import emoji
import re
import csv
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout, Input, CuDNNLSTM, CuDNNLSTM, RNN, LSTMCell,TimeDistributed, Flatten, Activation, RepeatVector, Permute, Lambda, multiply, Dot, Reshape, merge
from keras.models import Sequential, Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import time
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt


from tensorflow.keras import backend as K
from tensorflow.python.keras.utils import tf_utils


['glove-global-vectors-for-word-representation', 'emocontext']


Using TensorFlow backend.


In [2]:
path_prefix='../input/'
TRAIN_FILE = path_prefix + 'emocontext/train.txt'
DEV_FILE =  path_prefix + 'emocontext/dev.txt'
TEST_FILE =  path_prefix + 'emocontext/test.txt'

TURNS_NAMES = ["turn1", "turn2", "turn3"]
LABEL = ["label"]
TURNS_CONCAT = "turns"
def parse_file(file_path):
    output_dict = dict()
    with open(file_path, newline='\n', encoding='utf8') as csvfile:
        return pd.read_csv(csvfile, sep="\t")

train_data = parse_file(TRAIN_FILE)
dev_data = parse_file(DEV_FILE)
test_data = parse_file(TEST_FILE)

In [3]:
def concatenate_turns(df, delim="fullstop"):
    turns = [("%s %s %s %s %s" %
                 (row[TURNS_NAMES[0]], delim,
                  row[TURNS_NAMES[1]], delim,
                  row[TURNS_NAMES[2]])).lower()
                 for index, row in df.iterrows()]
    df[TURNS_CONCAT] = pd.Series(turns, index=df.index)
    return df
def emoticons_replace(df):
    for index, row in df.iterrows():
        for turn in range(3):
          turns = emoji.demojize(row[TURNS_NAMES[turn]])
          # remove delimiters ":"  (:smiley: -> smiley)
          for emoj in re.findall(":\w*:", turns):
              turns  = turns.replace(emoj, emoj[1:-1]).replace("_", " ")
          df.at[index, TURNS_NAMES[turn]] = turns
    return df

In [4]:
train = concatenate_turns(emoticons_replace(train_data))
dev = concatenate_turns(emoticons_replace(dev_data))
test = concatenate_turns(emoticons_replace(test_data))

In [5]:
tweet_tokenizer = TweetTokenizer()
def tokenize_turns(df):
    turns = [tweet_tokenizer.tokenize(row[TURNS_CONCAT]) 
                for idx, row in df.iterrows()]
    df[TURNS_CONCAT] = pd.Series(turns, index=df.index)
    return df
train_tok = tokenize_turns(train)
dev_tok = tokenize_turns(dev)
test_tok = tokenize_turns(test)

In [6]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tok[TURNS_CONCAT])
tokenizer.fit_on_texts(dev_tok[TURNS_CONCAT])
tokenizer.fit_on_texts(test_tok[TURNS_CONCAT])
VOCABULARY = tokenizer.word_index
VOCABULARY['unk'] = 0
vocabulary_size = len(VOCABULARY.keys()) + 1
max_sentence = 189 # 163,82,189
embed_dim = 200

In [7]:
glove_vectors_file = path_prefix+"glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt"

In [ ]:
glove_wordmap = {}
with open(glove_vectors_file, "r", encoding="utf8") as glove:
    for line in glove:
        name, vector = tuple(line.split(" ", 1))
        glove_wordmap[name] = np.fromstring(vector, sep=" ")

In [ ]:
embedding_matrix = np.zeros((vocabulary_size, embed_dim))
for word, i in VOCABULARY.items():
    embedding_vector = glove_wordmap.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
### angry: [1 0 0 0]
### happy: [0 1 0 0]
### others: [0 0 1 0]
### sad: [0 0 0 1]

labels = {0: 'angry',
          1: 'happy',
          2: 'others',
          3: 'sad'}
X_train = pad_sequences(tokenizer.texts_to_sequences(
                            train_tok[TURNS_CONCAT]),
                        maxlen=max_sentence)
X_dev = pad_sequences(tokenizer.texts_to_sequences(dev_tok[TURNS_CONCAT]), maxlen=max_sentence)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_tok[TURNS_CONCAT]), maxlen=max_sentence)
Y_train = pd.get_dummies(train[LABEL]).as_matrix()
Y_dev = pd.get_dummies(dev[LABEL]).as_matrix()
Y_test = pd.get_dummies(test[LABEL]).as_matrix()

### Metrics callback

In [ ]:
class Metrics(Callback):
    def __init__(self, test_X, test_Y, tolerance):
        self.test_X = test_X
        self.test_Y = test_Y
        self.max_f1 = 0
        self.f1_prev = 0
        self.tolerance = tolerance
        self.decreasing_times = 0
        
        
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self.i = 0
        self.x = []
        
        self.f1s_test = []
        self.f1s_val = []
        self.losses = []
        self.val_losses = []
        self.epoch=0
        
        self.logs = []
        self.fig = plt.figure()
    
    def plot_losses(self, f1_val,f1_test, logs):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.f1s_test.append(f1_test)
        self.f1s_val.append(f1_val)
        self.i += 1
        
#         clear_output(wait=True)
        
        plt.subplot(2,1,1)
        plt.plot(self.x, self.losses, label="train_loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.subplot(2,1,2)
        plt.plot(self.x, self.f1s_val, label="f1_val " + '{:.4f}'.format(max(self.f1s_val)))
        plt.plot(self.x, self.f1s_test, label="f1_test " + '{:.4f}, e={:d}'.format(max(self.f1s_test), self.epoch))
        
        plt.legend(loc=0)

        plt.show();

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]

        _val_f1 = f1_score(val_targ, val_predict, average='micro')
        _val_recall = recall_score(val_targ, val_predict, average='micro')
        _val_precision = precision_score(val_targ, val_predict, average='micro')

        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print("— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        predicts = self.model.predict(self.test_X)
        test_predict = (np.asarray(predicts)).round()
        f1 = print_metrics_predicted(val_predict, val_targ)
        print("#############\nF1 test:\n#############")
        f_test = print_metrics_predicted(test_predict, self.test_Y)
        self.plot_losses(f1, f_test, logs)
        if f_test > self.max_f1:
            self.max_f1 = f_test
            self.epoch = epoch
        if f_test < self.f1_prev:
            self.decreasing_times += 1
            if self.decreasing_times > self.tolerance:
                self.model.stop_training = True
        else:
            self.decreasing_times = 0
        self.f1_prev = f_test
        return
 
# metrics = Metrics()

def print_metrics_predicted(predicts,Y,filename=None):
    tp =[0,0,0,0]
    fp =[0,0,0,0]
    fn =[0,0,0,0]
    for i,pred in enumerate(predicts):
        p = np.argmax(pred)
        y = np.argmax(Y[i])
        if p == y:
            tp[p] += 1
        else:
            fp[p] +=1
            fn[y] +=1
    prec = sum(tp)/(sum(tp+fp)+np.finfo(float).eps)
    rec = sum(tp)/(sum(tp+fn)+np.finfo(float).eps)
    print("F1 all")
    f1_all = 2*prec*rec/(prec+rec+np.finfo(float).eps)
    print(f1_all) 
    print("***")
    for i in range(4):
      print("F1 %s: " % labels[i])
      prec = tp[i]/(tp[i]+fp[i]+np.finfo(float).eps)
      rec = tp[i]/(tp[i]+fn[i]+np.finfo(float).eps)
      f1 = 2*prec*rec/(prec+rec+np.finfo(float).eps)
      print(f1)
      print("****")
    tp.pop(2)
    fp.pop(2)
    fn.pop(2)
    print("F1 happy angry sad")
    prec = sum(tp)/(sum(tp+fp)+np.finfo(float).eps)
    rec = sum(tp)/(sum(tp+fn)+np.finfo(float).eps)
    f1= 2*prec*rec/(prec+rec+np.finfo(float).eps)
    print(f1)
    return f1


In [ ]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
 
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
 
        return context_vector, attention_weights

In [ ]:
epochs = 10
lstm_out = 128
batch_size = 128
drop_out = 0.3
loss_fct = 'binary_crossentropy'
activation_fct = 'softmax'
optimizer = "Adam-0.01"

### [Att seq2seq](https://github.com/neonbjb/ml-notebooks/blob/master/keras-seq2seq-with-attention/keras_translate_notebook.ipynb)

In [ ]:

# RNN "Cell" classes in Keras perform the actual data transformations at each timestep. Therefore, in order
# to add attention to LSTM, we need to make a custom subclass of LSTMCell.
class AttentionLSTMCell(LSTMCell):
    def __init__(self, **kwargs):
        self.attentionMode = False
        super(AttentionLSTMCell, self).__init__(**kwargs)
    
    # Build is called to initialize the variables that our cell will use. We will let other Keras
    # classes (e.g. "Dense") actually initialize these variables.
    @tf_utils.shape_type_conversion
    def build(self, input_shape):        
        # Converts the input sequence into a sequence which can be matched up to the internal
        # hidden state.
        self.dense_constant = TimeDistributed(Dense(self.units, name="AttLstmInternal_DenseConstant"))
        
        # Transforms the internal hidden state into something that can be used by the attention
        # mechanism.
        self.dense_state = Dense(self.units, name="AttLstmInternal_DenseState")
        
        # Transforms the combined hidden state and converted input sequence into a vector of
        # probabilities for attention.
        self.dense_transform = Dense(1, name="AttLstmInternal_DenseTransform")
        
        # We will augment the input into LSTMCell by concatenating the context vector. Modify
        # input_shape to reflect this.
        batch, input_dim = input_shape[0]
        batch, timesteps, context_size = input_shape[-1]
        lstm_input = (batch, input_dim + context_size)
        
        # The LSTMCell superclass expects no constant input, so strip that out.
        return super(AttentionLSTMCell, self).build(lstm_input)
    
    # This must be called before call(). The "input sequence" is the output from the 
    # encoder. This function will do some pre-processing on that sequence which will
    # then be used in subsequent calls.
    def setInputSequence(self, input_seq):
        self.input_seq = input_seq
        self.input_seq_shaped = self.dense_constant(input_seq)
        self.timesteps = tf.shape(self.input_seq)[-2]
    
    # This is a utility method to adjust the output of this cell. When attention mode is
    # turned on, the cell outputs attention probability vectors across the input sequence.
    def setAttentionMode(self, mode_on=False):
        self.attentionMode = mode_on
    
    # This method sets up the computational graph for the cell. It implements the actual logic
    # that the model follows.
    def call(self, inputs, states, constants):
        # Separate the state list into the two discrete state vectors.
        # ytm is the "memory state", stm is the "carry state".
        ytm, stm = states
        # We will use the "carry state" to guide the attention mechanism. Repeat it across all
        # input timesteps to perform some calculations on it.
        stm_repeated = K.repeat(self.dense_state(stm), self.timesteps)
        # Now apply our "dense_transform" operation on the sum of our transformed "carry state" 
        # and all encoder states. This will squash the resultant sum down to a vector of size
        # [batch,timesteps,1]
        # Note: Most sources I encounter use tanh for the activation here. I have found with this dataset
        # and this model, relu seems to perform better. It makes the attention mechanism far more crisp
        # and produces better translation performance, especially with respect to proper sentence termination.
        combined_stm_input = self.dense_transform(
            keras.activations.relu(stm_repeated + self.input_seq_shaped))
        # Performing a softmax generates a log probability for each encoder output to receive attention.
        score_vector = keras.activations.softmax(combined_stm_input, 1)
        # In this implementation, we grant "partial attention" to each encoder output based on 
        # it's log probability accumulated above. Other options would be to only give attention
        # to the highest probability encoder output or some similar set.
        context_vector = K.sum(score_vector * self.input_seq, 1)
        
        # Finally, mutate the input vector. It will now contain the traditional inputs (like the seq2seq
        # we trained above) in addition to the attention context vector we calculated earlier in this method.
        inputs = K.concatenate([inputs, context_vector])
        
        # Call into the super-class to invoke the LSTM math.
        res = super(AttentionLSTMCell, self).call(inputs=inputs, states=states)
        
        # This if statement switches the return value of this method if "attentionMode" is turned on.
        if(self.attentionMode):
            return (K.reshape(score_vector, (-1, self.timesteps)), res[1])
        else:
            return res

# Custom implementation of the Keras LSTM that adds an attention mechanism.
# This is implemented by taking an additional input (using the "constants" of the
# RNN class) into the LSTM: The encoder output vectors across the entire input sequence.
class LSTMWithAttention(RNN):
    def __init__(self, units, **kwargs):
        cell = AttentionLSTMCell(units=units)
        self.units = units
        super(LSTMWithAttention, self).__init__(cell, **kwargs)
        
    @tf_utils.shape_type_conversion
    def build(self, input_shape):
        self.input_dim = input_shape[0][-1]
        self.timesteps = input_shape[0][-2]
        return super(LSTMWithAttention, self).build(input_shape) 
    
    # This call is invoked with the entire time sequence. The RNN sub-class is responsible
    # for breaking this up into calls into the cell for each step.
    # The "constants" variable is the key to our implementation. It was specifically added
    # to Keras to accomodate the "attention" mechanism we are implementing.
    def call(self, x, constants, **kwargs):
        if isinstance(x, list):
            self.x_initial = x[0]
        else:
            self.x_initial = x
        
        # The only difference in the LSTM computational graph really comes from the custom
        # LSTM Cell that we utilize.
        self.cell._dropout_mask = None
        self.cell._recurrent_dropout_mask = None
        self.cell.setInputSequence(constants[0])
        return super(LSTMWithAttention, self).call(inputs=x, constants=constants, **kwargs)

In [ ]:
# units = 512
# def create_model():
#     input_layer = Input(shape=(max_sentence,), dtype='int32')

#     embedding_layer = Embedding(vocabulary_size,
#                                 embed_dim,
#                                 weights=[embedding_matrix],
#                                 input_length=max_sentence,
#                                 trainable=True)(input_layer)
#     attenc_outputs, attstate_h, attstate_c = CuDNNLSTM(units=units, return_sequences=True, return_state=True)(embedding_layer)
#     attenc_states = [attstate_h, attstate_c]
    
#     attdec_inputs = Input(shape=(None,))
#     attdec_emb = Embedding(vocabulary_size,
#                                 embed_dim,
#                                 weights=[embedding_matrix],
#                                 input_length=max_sentence,
#                                 trainable=True)(attdec_inputs)
#     attdec_lstm = LSTMWithAttention(units=units, return_sequences=True, return_state=True)
#     # Note that the only real difference here is that we are feeding attenc_outputs to the decoder now.
#     attdec_lstm_out, _, _ = attdec_lstm(inputs=attdec_emb, 
#                                         constants=attenc_outputs, 
#                                         initial_state=attenc_states)
    
    
#     attdec_d1 = Dense(128, activation="relu")
#     attdec_d2 = Dense(4, activation="softmax")
#     attdec_out = attdec_d2(Dropout(rate=.4)(attdec_d1(Dropout(rate=.4)(attdec_lstm_out))))
    
    
#     attmodel = Model(input_layer, attdec_out)
#     attmodel.compile(optimizer=tf.train.AdamOptimizer(), loss="sparse_categorical_crossentropy", metrics=['sparse_categorical_accuracy'])
    
#     return attmodel
    
#     bi_lstm = Bidirectional(LSTM(256))(embedding_layer)
#     dropout = Dropout(0.4)(bi_lstm)
#     dense = Dense(128,activation='relu')(dropout)
#     dropout = Dropout(0.2)(dense)
#     dense = Dense(64,activation='relu')(dropout)
#     dropout = Dropout(0.2)(dense)
#     dense = Dense(4,activation='softmax')(dropout)
#     adam = optimizers.Adam(lr=0.01)
#     rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
#     sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

#     model = Model(inputs=[input_layer], outputs=dense)
#     model.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy'])
#     return model

### [Attention 3d block](https://github.com/philipperemy/keras-attention-mechanism/blob/master/attention_lstm.py)

In [ ]:


def get_activations(model, inputs, print_shape_only=False, layer_name=None):
    # Documentation is available online on Github at the address below.
    # From: https://github.com/philipperemy/keras-visualize-activations
    print('----- activations -----')
    activations = []
    inp = model.input
    if layer_name is None:
        outputs = [layer.output for layer in model.layers]
    else:
        outputs = [layer.output for layer in model.layers if layer.name == layer_name]  # all layer outputs
    funcs = [K.function([inp] + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions
    layer_outputs = [func([inputs, 1.])[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)
        if print_shape_only:
            print(layer_activations.shape)
        else:
            print(layer_activations)
    return activations


def get_data(n, input_dim, attention_column=1):
    """
    Data generation. x is purely random except that it's first value equals the target y.
    In practice, the network should learn that the target = x[attention_column].
    Therefore, most of its attention should be focused on the value addressed by attention_column.
    :param n: the number of samples to retrieve.
    :param input_dim: the number of dimensions of each element in the series.
    :param attention_column: the column linked to the target. Everything else is purely random.
    :return: x: model inputs, y: model targets
    """
    x = np.random.standard_normal(size=(n, input_dim))
    y = np.random.randint(low=0, high=2, size=(n, 1))
    x[:, attention_column] = y[:, 0]
    return x, y


def get_data_recurrent(n, time_steps, input_dim, attention_column=10):
    """
    Data generation. x is purely random except that it's first value equals the target y.
    In practice, the network should learn that the target = x[attention_column].
    Therefore, most of its attention should be focused on the value addressed by attention_column.
    :param n: the number of samples to retrieve.
    :param time_steps: the number of time steps of your series.
    :param input_dim: the number of dimensions of each element in the series.
    :param attention_column: the column linked to the target. Everything else is purely random.
    :return: x: model inputs, y: model targets
    """
    x = np.random.standard_normal(size=(n, time_steps, input_dim))
    y = np.random.randint(low=0, high=2, size=(n, 1))
    x[:, attention_column, :] = np.tile(y[:], (1, input_dim))
    return x, y

In [ ]:
INPUT_DIM = 2
TIME_STEPS = max_sentence
# if True, the attention vector is shared across the input_dimensions where the attention is applied.
SINGLE_ATTENTION_VECTOR = True
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul

In [ ]:
lstm_units = 1024
def create_model():
    input_layer = Input(shape=(max_sentence,), dtype='int32')
    embedding_layer = Embedding(vocabulary_size,
                                embed_dim,
                                weights=[embedding_matrix],
                                input_length=max_sentence,
                                trainable=True)(input_layer)  
    
#     attention_mul = attention_3d_block(embedding_layer)
#     bi_lstm = Bidirectional(LSTM(256,  return_sequences=False))(embedding_layer)
    
    lstm_out = LSTM(lstm_units, return_sequences=True)(embedding_layer)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)


    
#     dropout = Dropout(0.3)(attention_mul)

#     dropout = Dropout(0.3)(attention_mul)
    dense = Dense(256,activation='relu')(attention_mul)
    dropout = Dropout(0.2)(dense)
    dense = Dense(128,activation='relu')(dropout)
    dropout = Dropout(0.2)(dense)
    dense = Dense(4, activation='softmax')(dropout)



    
#     dense = Dense(4,activation='softmax')(dropout)
    adam = optimizers.Adam(lr=0.01)
    rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
#     sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    model = Model(inputs=input_layer, outputs=dense)
    model.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy'])
    return model  


In [ ]:
model = create_model()
model.summary()

In [ ]:
import keras
import IPython
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
keras.utils.plot_model(model, to_file='test_keras_plot_model.png', show_shapes=True,show_layer_names=False)
IPython.display.Image('test_keras_plot_model.png')

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])

#### Attention tanh - [link](https://github.com/keras-team/keras/issues/4962)0.7044

* * #############
* F1 all
* 0.9045198765656198
* ***
* F1 angry: 
* 0.6973500697350068
* ****
* F1 happy: 
* 0.6750788643533122
* ****
* F1 others: 
* 0.9455380577427821
* ****
* F1 sad: 
* 0.7495219885277247
* ****
* F1 happy angry sad
* 0.7043756670224119

In [ ]:
units = 512
def create_model():
    input_layer = Input(shape=(max_sentence,), dtype='int32')
    embedding_layer = Embedding(vocabulary_size,
                                embed_dim,
                                weights=[embedding_matrix],
                                input_length=max_sentence,
                                trainable=False)(input_layer)  
#     bi_lstm = Bidirectional(LSTM(256))(embedding_layer)
    
    
    activations = LSTM(units, return_sequences=True)(embedding_layer)
    
    attention = TimeDistributed(Dense(1, activation='tanh'))(activations) 
    attention = Dense(1, activation='tanh')(activations) 
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(units)(attention)
    attention = Permute([2, 1])(attention)

    # apply the attention
#     sent_representation = multiply([activations, attention])
#     sent_representation = Lambda(lambda xin: K.sum(xin, axis=0), output_shape=(units, ))(sent_representation)
#     sent_representation = Flatten()(sent_representation)



#     sent_representation = multiply([activations, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=0), output_shape=(units, ))(attention)


    dense = Dense(4, activation='softmax')(sent_representation)



    
#     dropout = Dropout(0.4)(bi_lstm)
#     dense = Dense(128,activation='relu')(dropout)
#     dropout = Dropout(0.2)(dense)
#     dense = Dense(64,activation='relu')(dropout)
#     dropout = Dropout(0.2)(dense)
#     dense = Dense(4,activation='softmax')(dropout)
    
    adam = optimizers.Adam(lr=0.01)
    rmsprop = optimizers.RMSprop(lr=0.005)#, rho=0.9, epsilon=None, decay=0.0)
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    model = Model(inputs=input_layer, outputs=dense)
    model.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy'])
    return model  


In [ ]:
model = create_model()
model.summary()

In [ ]:
import keras
import IPython
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
keras.utils.plot_model(model, to_file='test_keras_plot_model.png', show_shapes=True,show_layer_names=False)
IPython.display.Image('test_keras_plot_model.png')

In [ ]:
import keras
import IPython
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
keras.utils.plot_model(model, to_file='test_keras_plot_model.png', show_shapes=True,show_layer_names=False)
IPython.display.Image('test_keras_plot_model.png') # deep lstm

In [ ]:
import keras
import IPython
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
keras.utils.plot_model(model, to_file='test_keras_plot_model.png', show_shapes=True,show_layer_names=False)
IPython.display.Image('test_keras_plot_model.png')

### Deep bilstm diagram

In [ ]:
import keras
import IPython
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
keras.utils.plot_model(model, to_file='test_keras_plot_model.png', show_shapes=True,show_layer_names=False)
IPython.display.Image('test_keras_plot_model.png') # deep bilstm

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])# att + deep lstm 

In [ ]:
import keras
import IPython
# SVG(model_to_dot(model).create(prog='dot', format='svg'))
keras.utils.plot_model(model, to_file='test_keras_plot_model.png', show_shapes=True,show_layer_names=False)
IPython.display.Image('test_keras_plot_model.png') # deep bilstm + single vector

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])# deep bilstm + single vector+ rmsprop

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])# deep bilstm + single vector+ adam

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])# att+deep bilstm

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])# deep lstm

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])# adam + categorical

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ])# adam + binary

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=189,
          validation_data=(X_dev, Y_dev),
          callbacks=[Metrics(X_test, Y_test, 3),
                    ]) #rmsprop